In [5]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt

# 데이터 다운로드
btc = yf.download('BTC-USD', start='2020-01-01', end='2023-06-22')

# 기술적 지표 계산
def add_indicators(df):
    df['SMA_7'] = df['Close'].rolling(window=7).mean()
    df['SMA_30'] = df['Close'].rolling(window=30).mean()
    df['RSI'] = calculate_rsi(df['Close'], 14)
    df['MACD'], df['Signal'] = calculate_macd(df['Close'])
    return df

def calculate_rsi(prices, period=14):
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_macd(prices, slow=26, fast=12, signal=9):
    exp1 = prices.ewm(span=fast, adjust=False).mean()
    exp2 = prices.ewm(span=slow, adjust=False).mean()
    macd = exp1 - exp2
    signal = macd.ewm(span=signal, adjust=False).mean()
    return macd, signal

btc = add_indicators(btc)
btc.dropna(inplace=True)

# 특성 선택
features = ['Open', 'High', 'Low', 'Close', 'Volume', 'SMA_7', 'SMA_30', 'RSI', 'MACD', 'Signal']
X = btc[features].values
y = btc['Close'].values

# 데이터 정규화
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y.reshape(-1, 1))

# 시계열 데이터 준비
def create_sequences(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

time_steps = 60
X_seq, y_seq = create_sequences(X_scaled, y_scaled, time_steps)

# 훈련 및 테스트 세트 분할
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

# LSTM 모델 구축
model = Sequential([
    LSTM(50, activation='relu', input_shape=(time_steps, X.shape[1]), return_sequences=True),
    Dropout(0.2),
    LSTM(50, activation='relu', return_sequences=False),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# 모델 훈련
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

# 예측
y_pred = model.predict(X_test)

# 결과 시각화
plt.figure(figsize=(12, 6))
plt.plot(scaler.inverse_transform(y_test), label='Actual Price')
plt.plot(scaler.inverse_transform(y_pred), label='Predicted Price')
plt.title('Bitcoin Price Prediction')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()

# 모델 성능 평가
from sklearn.metrics import mean_squared_error, mean_absolute_error
mse = mean_squared_error(scaler.inverse_transform(y_test), scaler.inverse_transform(y_pred))
mae = mean_absolute_error(scaler.inverse_transform(y_test), scaler.inverse_transform(y_pred))
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')

[*********************100%%**********************]  1 of 1 completed

Accuracy: 0.41
